# `--- Day 19: A Series of Tubes ---`

In [1]:
import operator
from functools import reduce

In [2]:
test_map = '''     |          
     |  +--+    
     A  |  C    
 F---|----E|--+ 
     |  |  |  D 
     +B-+  +--+ '''.split('\n')

real_map = list(map(lambda x: x.strip('\n'), open('input.txt').readlines()))

In [3]:
# (0,0) is top left, coordinates are (row, column)
DELTA = {'N': (-1,  0),
         'E': ( 0,  1),
         'S': ( 1,  0),
         'W': ( 0, -1)}

DIRECTIONS = set(DELTA.keys())
# which way to turn...
ORTHOGONAL = {'N': 'EW',
              'S': 'EW',
              'E': 'NS',
              'W': 'NS'}
# in case part 2 is counting self-crossings
CROSSINGS = {'N': '-',
             'E': '|',
             'S': '-',
             'W': '|'}
# the path
PIPES = {'N': '|',
         'E': '-',
         'S': '|',
         'W': '-'}

ALPHA = list('ABCDEFGHIJKLMNOPQRSTUVWXYZ')

BORDER = '#'

class MapWalker(object):
    
    def __init__(self, maplines):
        # add a border of "#" around the map so we don't have to worry about bounds checking
        l = len(maplines[0]) + 2
        extended_maplines = [list(BORDER * l)] + [[BORDER] + list(x) + [BORDER] for x in maplines] + [list(BORDER * l)]
        self.map = extended_maplines
        
        self.trackchars = set(reduce(operator.add, self.map))
        self.trackchars.remove(' ') # works in place
        self.trackchars.remove('#') # works in place
        self.row = None
        self.col = None
        self.height = len(self.map)
        self.width = len(self.map[0])
        self.direction = None
        self.unders = 0 # number of times we go under another line
        self.turns = 0
        self.steps = 0
        self.letters = ''
        
    def initialise(self):
        '''Find the start position on the North side'''
        self.row = 1
        for i, c in enumerate(self.map[self.row]):
            if c in self.trackchars:
                self.col = i
        self.direction = 'S'
        return self
    
    def at(self, row, col):
        return self.map[row][col]
    
    def here(self):
        return self.at(self.row, self.col)
    
    def look(self, direction):
        '''character at the curretn location plus one step in the given direction'''
        dr, dc = DELTA[direction]
        return self.map[self.row + dr][self.col + dc]
    
    def step(self):
        
        self.steps += 1
        
        drc = DELTA[self.direction]
        
        self.row = self.row + drc[0]
        self.col = self.col + drc[1]
        
        here = self.here()
        
        if here in ALPHA:
            self.letters += self.here()
        elif here == PIPES[self.direction]:
            # carry on - just a continued pipe
            pass
        elif here == '+':
            self.turns += 1
            dirns = ORTHOGONAL[self.direction]
            for d in dirns:
                if self.look(d) == PIPES[d] or self.look(d) in ALPHA:
                    self.direction = d
                    break
        elif here == CROSSINGS[self.direction]:
            self.unders += 1
        else:
            print('finished!')
            print(f'letters: {self.letters}, steps {self.steps}, unders: {self.unders}, turns: {self.turns}')
            raise StopIteration()
        return self
    
    def walk(self):
        self.initialise()
        while True:
            try:
                self.step()
            except StopIteration:
                break
        return self
           
    def __str__(self):
        _str = ''
        for row, l in enumerate(self.map):
            for col, c in enumerate(l):
                #print(c)
                _str += '@' if (row, col) == (self.row, self.col) else c
            _str += '\n'
        return _str
    
    __repr__ = __str__

# Test

In [4]:
foo = MapWalker(test_map)

foo.walk()

finished!
letters: ABCDEF, steps 38, unders: 3, turns: 7


##################
#     |          #
#     |  +--+    #
#     A  |  C    #
#@F---|----E|--+ #
#     |  |  |  D #
#     +B-+  +--+ #
##################

# Real

In [5]:
real = MapWalker(real_map)

_ = real.walk()

finished!
letters: HATBMQJYZ, steps 16332, unders: 1486, turns: 455
